# 25.06.30 LangChain Challenge Start

with [Nomad Coder](https://nomadcoders.co/c/gpt-challenge/lobby)

In [17]:
from IPython.terminal.shortcuts.filters import pass_through
from dotenv import load_dotenv
import os
from pathlib import Path
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotPromptTemplate, PromptTemplate
from langchain.schema import StrOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain, SimpleSequentialChain, ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough

raw = Path(".env").read_bytes()
load_dotenv(dotenv_path=Path(".env"), override=True)

assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY is not set"
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),]
)

In [4]:
poem_prompt = ChatPromptTemplate.from_template(
    "다음 프로그래밍 언어에 대한 짧은 시를 영어로 지어줘. 그리고 한글로 번역도 작성 해줘.:\n\n"
    "언어: {language}\n\n"
    "시:"
)

poem_chain = LLMChain(llm=model, prompt=poem_prompt, output_key="poem")

explain_prompt = ChatPromptTemplate.from_template(
    "아래 시를 보고, 콜롬비아 출신의 사람에게 딱 맞는 비유와 은유를 들어 영어로 설명 해줘:\n\n"
    "{poem}\n\n"
    "설명:"
)

explain_chain = LLMChain(llm=model, prompt=explain_prompt, output_key="explanation")

full_chain = poem_chain | explain_chain
full_chain.invoke({"language": "Python"})

# seq_chain = SimpleSequentialChain(
#     chains=[poem_chain, explain_chain],
#     input_key='language',
#     output_key='explanation',
#     verbose=True
# )
#
# result = seq_chain.run({"language": "Python"})

In the world of code, Python slithers with grace,
With its clean syntax and powerful embrace.
From data analysis to web apps so fine,
Python's versatility truly shines.

한글 번역:
코드의 세계에서, 파이썬은 우아하게 기어다닙니다,
깔끔한 구문과 강력한 기능을 갖추고 있죠.
데이터 분석부터 멋진 웹 앱까지,
파이썬의 다재다능함이 찬란히 빛나네요.In the world of code, Python is like a skilled salsa dancer from Colombia, moving with grace and precision. Just like Colombian dancers are known for their fluid movements and elegance, Python is known for its clean syntax and powerful capabilities. Much like how Colombian dancers can seamlessly transition between different styles of dance, Python can be used for a wide range of tasks such as data analysis and creating web applications. Just as Colombian culture is rich and diverse, Python's versatility truly shines in the world of programming.

{'language': 'Python',
 'poem': "In the world of code, Python slithers with grace,\nWith its clean syntax and powerful embrace.\nFrom data analysis to web apps so fine,\nPython's versatility truly shines.\n\n한글 번역:\n코드의 세계에서, 파이썬은 우아하게 기어다닙니다,\n깔끔한 구문과 강력한 기능을 갖추고 있죠.\n데이터 분석부터 멋진 웹 앱까지,\n파이썬의 다재다능함이 찬란히 빛나네요.",
 'explanation': "In the world of code, Python is like a skilled salsa dancer from Colombia, moving with grace and precision. Just like Colombian dancers are known for their fluid movements and elegance, Python is known for its clean syntax and powerful capabilities. Much like how Colombian dancers can seamlessly transition between different styles of dance, Python can be used for a wide range of tasks such as data analysis and creating web applications. Just as Colombian culture is rich and diverse, Python's versatility truly shines in the world of programming."}

# 25.07.01 Challenge

In [42]:
examples = [
    {
        'question': '영화 갓 파더에 대해 설명해줘.',
        'answer': """
            '감독': '프랑시스 포드 코폴라',
            '주요_출연진': ['Marlon Brando', 'Al Pacino', 'James Caan'],
            '예산': 6000000,
            '흥행_수익': 246120974,
            '장르': '범죄',
            '간략한_줄거리': 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.'
        """
    },
    {
        'question': '오징어 게임에 대해 설명 해줘.',
        'answer': """
            '감독': '황동혁',
            '주요_출연진': ['이정재', '타노스', '이병헌'],
            '예산': 100000,
            '흥행_수익': 423413491,
            '장르': '드라마',
            '간략한_줄거리': '어느날 사람들이 의문의 게임에 초대받고, 생존을 위해 목숨을 건 게임에 참여하게 된다. 이 게임은 상금이 걸린 치열한 경쟁으로, 참가자들은 각자의 사연과 이유로 게임에 참여하게 된다.'
        """
    },
    {
        'question': '인디아나 존스 영화에 대해서 설명 해줘.',
        'answer': """
            '감독': '스티븐 스필버그',
            '주요_출연진': ['Harrison Ford', 'Karen Allen', 'Paul Freeman'],
            '예산': 18000000,
            '흥행_수익': 389925971,
            '장르': '어드벤처',
            '간략한_줄거리': 'An archaeologist is hired by the U.S. government to find the Ark of the Covenant.'
        """
    },
    {
        'question': '영화 타이타닉에 대해서 설명 해줘.',
        'answer': """
            '감독': '제임스 카메론',
            '주요_출연진': ['Kate Winslet', 'Leonardo DiCaprio', 'Billy Zane'],
            '예산': 200000000,
            '흥행_수익': 2187463944,
            '장르': '로맨스',
            '간략한_줄거리': 'A seventeen-year-old aristocrat falls in love with a kind but poor artist aboard the luxurious, ill-fated R.M.S. Titanic.'
        """
    }
]

In [48]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}"
)
# print(example_prompt.format(**examples[0]))

In [49]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}\nAnswer:",
    input_variables=["input"],
)
# print(prompt.format(input="오징어 게임"))

In [50]:
chain = LLMChain(llm=model, prompt=prompt, output_key="answer")

chain.invoke({"input": "마더"})

'감독': '봉준호',
'주요_출연진': ['김혜자', '원빈', '진구'],
'예산': 5000000,
'흥행_수익': 16917651,
'장르': '스릴러',
'간략한_줄거리': '실종된 딸을 찾기 위해 모든 것을 걸고 싸우는 엄마의 이야기를 그린 영화.'

{'input': '마더',
 'answer': "'감독': '봉준호',\n'주요_출연진': ['김혜자', '원빈', '진구'],\n'예산': 5000000,\n'흥행_수익': 16917651,\n'장르': '스릴러',\n'간략한_줄거리': '실종된 딸을 찾기 위해 모든 것을 걸고 싸우는 엄마의 이야기를 그린 영화.'"}

# 25.07.02 Challenge

In [111]:
examples = [
    {
        'question': '탑건',
        'answer': "🛩️👨‍✈️🔥"
    },
    {
        'question': '마더',
        'answer': "🙎🏻‍♀️🔫🍝"
    },
    {
        'question': '오징어게임',
        'answer': "🦑🎮💰"
    },
    {
        'question': '타이타닉',
        'answer': "🚢💔🌊"
    },
    {
        'question': '인디아나 존스',
        'answer': "🏺🗺🧳"
    },
    {
        'question': '아이언맨',
        'answer': "🤖🦾💥"
    }
]

In [112]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}"
)
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="History:\n{history}\nQuestion: {input}\nAnswer:",
    input_variables=["history", "input"],
)

In [113]:
memory = ConversationBufferMemory(
    memory_key='history',
    input_key='input',
    return_messages=True
)

In [114]:
def load_memory(_):
    print(_)
    return memory.load_memory_variables({})['history']

chain = RunnablePassthrough.assign(history=load_memory) | prompt | model

def invoke_chain(question):
    result = chain.invoke({
        "input": question
    })
    memory.save_context(
        {"input": question},
        {"answer": result.content}
    )
    print("\n>>\n", memory)

In [115]:
invoke_chain("헝거 게임")

{'input': '헝거 게임'}
🏹🔥🏆
>>
 chat_memory=ChatMessageHistory(messages=[HumanMessage(content='헝거 게임'), AIMessage(content='🏹🔥🏆')]) input_key='input' return_messages=True


In [116]:
invoke_chain("닥터스트레인지")

{'input': '닥터스트레인지'}
🔮👨‍⚕️🌀
>>
 chat_memory=ChatMessageHistory(messages=[HumanMessage(content='헝거 게임'), AIMessage(content='🏹🔥🏆'), HumanMessage(content='닥터스트레인지'), AIMessage(content='🔮👨\u200d⚕️🌀')]) input_key='input' return_messages=True


In [117]:
invoke_chain("내가 마지막에 무엇을 물어봤지?")

{'input': '내가 마지막에 무엇을 물어봤지?'}
 내가 마지막에 무엇을 물어봤지? 닥터스트레인지
>>
 chat_memory=ChatMessageHistory(messages=[HumanMessage(content='헝거 게임'), AIMessage(content='🏹🔥🏆'), HumanMessage(content='닥터스트레인지'), AIMessage(content='🔮👨\u200d⚕️🌀'), HumanMessage(content='내가 마지막에 무엇을 물어봤지?'), AIMessage(content=' 내가 마지막에 무엇을 물어봤지? 닥터스트레인지')]) input_key='input' return_messages=True


In [118]:
invoke_chain("내가 처음에 무슨 영화를 물어봤지?")

{'input': '내가 처음에 무슨 영화를 물어봤지?'}
탑건
>>
 chat_memory=ChatMessageHistory(messages=[HumanMessage(content='헝거 게임'), AIMessage(content='🏹🔥🏆'), HumanMessage(content='닥터스트레인지'), AIMessage(content='🔮👨\u200d⚕️🌀'), HumanMessage(content='내가 마지막에 무엇을 물어봤지?'), AIMessage(content=' 내가 마지막에 무엇을 물어봤지? 닥터스트레인지'), HumanMessage(content='내가 처음에 무슨 영화를 물어봤지?'), AIMessage(content='탑건')]) input_key='input' return_messages=True


# 25.07.04 Challenge
Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
체인을 수동으로 구현해야 합니다.
체인에 ConversationBufferMemory를 부여합니다.
이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
체인에 다음 질문을 합니다:
Aaronson 은 유죄인가요?
그가 테이블에 어떤 메시지를 썼나요?
Julia 는 누구인가요?

In [40]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.storage import InMemoryStore

# Step 1: 문서 로드
def load_document():
    loader = TextLoader('./text.txt')
    documents = loader.load()
    return documents

# Step 2: 문서 분할
def split_documents(documents):
    splitter = CharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separator="\n"
    )
    return splitter.split_documents(documents)

# Step 3: 임베딩 및 캐시 설정
def create_embeddings():
    # 기본 임베딩 모델
    base_embeddings = OpenAIEmbeddings()
    # 캐시 저장소 생성
    store = InMemoryStore()
    # 캐시된 임베딩 생성
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
        base_embeddings,
        store,
        namespace="openai_embeddings"
    )
    return cached_embeddings

# Step 4: 벡터 스토어 생성
def create_vector_store(documents, embeddings):
    return FAISS.from_documents(documents, embeddings)

# Step 5: 대화 메모리 설정
def create_memory():
    return ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key="answer"
    )

# Step 6: RAG 체인 생성
def create_rag_chain(vectorstore, memory):
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
        temperature=0
    )
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
        memory=memory,
        return_source_documents=True
    )
    return chain

print("1. 문서 로드 중...")
documents = load_document()

print("2. 문서 분할 중...")
chunks = split_documents(documents)
print(f"총 {len(chunks)}개의 청크로 분할됨")

print("3. 임베딩 생성 중...")
embeddings = create_embeddings()

print("4. 벡터 스토어 생성 중...")
vectorstore = create_vector_store(chunks, embeddings)

print("5. 메모리 및 체인 설정 중...")
memory = create_memory()
chain = create_rag_chain(vectorstore, memory)

1. 문서 로드 중...
2. 문서 분할 중...
총 79개의 청크로 분할됨
3. 임베딩 생성 중...
4. 벡터 스토어 생성 중...
5. 메모리 및 체인 설정 중...


In [41]:
chain.invoke('Aaronson 은 유죄인가요?')

네, Jones, Aaronson, 그리고 Rutherford는 자신들에게 죄를 물려주었다고 기소된 것이 맞습니다.

{'question': 'Aaronson 은 유죄인가요?',
 'chat_history': [HumanMessage(content='Aaronson 은 유죄인가요?'),
  AIMessage(content='네, Jones, Aaronson, 그리고 Rutherford는 자신들에게 죄를 물려주었다고 기소된 것이 맞습니다.')],
 'answer': '네, Jones, Aaronson, 그리고 Rutherford는 자신들에게 죄를 물려주었다고 기소된 것이 맞습니다.',
 'source_documents': [Document(page_content='the thoughts that came into his head. He wrote first in large clumsy\ncapitals:\nFREEDOM IS SLAVERY\nThen almost without a pause he wrote beneath it:\nTWO AND TWO MAKE FIVE\nBut then there came a sort of check. His mind, as though shying away from\nsomething, seemed unable to concentrate. He knew that he knew what came\nnext, but for the moment he could not recall it. When he did recall it,\nit was only by consciously reasoning out what it must be: it did not come\nof its own accord. He wrote:\nGOD IS POWER\nHe accepted everything. The past was alterable. The past never had been\naltered. Oceania was at war with Eastasia. Oceania had always been at war\nwith Eastasia. Jones, Aaronso

In [42]:
chain.invoke('그가 테이블에 어떤 메시지를 썼나요?')

그가 테이블에 어떤 메시지를 썼습니까?그가 테이블에 쓴 메시지는 "FREEDOM IS SLAVERY"와 "TWO AND TWO MAKE FIVE" 그리고 "GOD IS POWER"입니다.

{'question': '그가 테이블에 어떤 메시지를 썼나요?',
 'chat_history': [HumanMessage(content='Aaronson 은 유죄인가요?'),
  AIMessage(content='네, Jones, Aaronson, 그리고 Rutherford는 자신들에게 죄를 물려주었다고 기소된 것이 맞습니다.'),
  HumanMessage(content='그가 테이블에 어떤 메시지를 썼나요?'),
  AIMessage(content='그가 테이블에 쓴 메시지는 "FREEDOM IS SLAVERY"와 "TWO AND TWO MAKE FIVE" 그리고 "GOD IS POWER"입니다.')],
 'answer': '그가 테이블에 쓴 메시지는 "FREEDOM IS SLAVERY"와 "TWO AND TWO MAKE FIVE" 그리고 "GOD IS POWER"입니다.',
 'source_documents': [Document(page_content='impossible even to rise from the horizontal if it had not been for the\nbottle and teacup placed beside the bed overnight. Through the midday\nhours he sat with glazed face, the bottle handy, listening to the\ntelescreen. From fifteen to closing-time he was a fixture in the Chestnut\nTree. No one cared what he did any longer, no whistle woke him, no\ntelescreen admonished him. Occasionally, perhaps twice a week, he went\nto a dusty, forgotten-looking office in the Ministry of Truth and did\na little work, or

In [43]:
chain.invoke('Julia 는 누구인가요?')

Julia는 누구인가요?Julia는 이야기 속에서 주인공인 윈스턴과 사랑을 나누는 여성 캐릭터입니다.

{'question': 'Julia 는 누구인가요?',
 'chat_history': [HumanMessage(content='Aaronson 은 유죄인가요?'),
  AIMessage(content='네, Jones, Aaronson, 그리고 Rutherford는 자신들에게 죄를 물려주었다고 기소된 것이 맞습니다.'),
  HumanMessage(content='그가 테이블에 어떤 메시지를 썼나요?'),
  AIMessage(content='그가 테이블에 쓴 메시지는 "FREEDOM IS SLAVERY"와 "TWO AND TWO MAKE FIVE" 그리고 "GOD IS POWER"입니다.'),
  HumanMessage(content='Julia 는 누구인가요?'),
  AIMessage(content='Julia는 이야기 속에서 주인공인 윈스턴과 사랑을 나누는 여성 캐릭터입니다.')],
 'answer': 'Julia는 이야기 속에서 주인공인 윈스턴과 사랑을 나누는 여성 캐릭터입니다.',
 'source_documents': [Document(page_content="out, reconciled. There were no more doubts, no more arguments, no more\npain, no more fear. His body was healthy and strong. He walked easily,\nwith a joy of movement and with a feeling of walking in sunlight. He was\nnot any longer in the narrow white corridors in the Ministry of Love, he\nwas in the enormous sunlit passage, a kilometre wide, down which he had\nseemed to walk in the delirium induced by drugs. He was in the Golden\nCountry, foll